# Freight Pricing API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = 'Not Valid' #replace with your subscription key

## Calculating a freight price
The Freight pricing API Retrieves freight prices for moving commodities between two port.
The following function is available:

1. get_freight_pricing.

Args:

    vessel_type: The type of vessel to calculate the prices for.
    load_port: Port where the commodity is loaded.
    discharge_port: Port where the commodity is discharged.
    date: Date at which the freight price is requested.
    vessel_subclass: The vessel's subclass.

Returns:

    A collection of freight pricing items, one per vessel class.
    
### Moving commodity from Fujairah towards Singapore
The following example will display how to calculate the available freight prices to move a commodity from Fujairah to Singapore.
We will use also the PortAPI to map ports to the Signal Geo schema.

In [2]:
from signal_ocean import Connection
connection = Connection(api_key=signal_ocean_api_key)

### Call the freight pricing API
Now that we have the ports we want to query about, we can call the freight pricing API and get out the commodity transfer cost.

In [3]:
from signal_ocean.freight_pricing import FreightPricingAPI, VesselTypeFilter, VesselClassFilter, PortFilter, VesselType, VesselSubclass
from datetime import date

freight_pricing_api = FreightPricingAPI(connection)

vessel_type = freight_pricing_api.get_vessel_types(VesselTypeFilter(name_like='tanker'))[0]
load_port = freight_pricing_api.get_ports(PortFilter(name_like='Fujairah'))[0]
discharge_port = freight_pricing_api.get_ports(PortFilter(name_like='Singapore'))[0]

today = date.today()

freight_pricing = freight_pricing_api.get_freight_pricing(
    vessel_type,
    load_port=load_port, 
    discharge_port=discharge_port,
    date=today,
    vessel_subclass=VesselSubclass.DIRTY
)

### Print out the result

Now we can hold the result and print it out on a dataframe

In [4]:
import pandas as pd

dataf = pd.DataFrame([{
    'vessel_class': p.vessel_class,
    'cargo_quantity': p.cargo_quantity,
    **p.costs.__dict__,
    **p.totals.__dict__ } for p in freight_pricing
])

dataf

,vessel_class,cargo_quantity,freight_rate,freight_cost,canal,total_cost,total_cost_per_ton
0,VLCC,270000.0,3.90558,1054506.6,0.0,1054506.6,3.90558
1,Suezmax,130000.0,5.402719,702353.47,0.0,702353.47,5.402719
2,Aframax,80000.0,7.16023,572818.4,0.0,572818.4,7.16023
